In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from numpy import fft
%matplotlib inline

## Statistics

### 1

Assuming there is actually no effect of the marketing campaign on sales, we would expect to see this or more extreme data less than 5% of the time. 

### 2

Assuming the null is correct, I would expect to get a p-value less than 0.05 5 times. The probability of the null actually being correct (conditional on the data of the first subsample) is 0.05 (probability of a type 1 error). 

The probability the null is correctly rejected based on the first test is 0.95 (1- the prob. of a type 1 error). Conditional on the null not being correct, the probability it is not rejected in subsequent tests is equal to the probability of a type 2 error (which I'll denote by beta_i, where i corresponds to the subsample). Since I don't know beta_i for all i (though it would be affected by factors such as sample size), it would be used to calculate the expected number of times the test stat is less than 0.05. After arriving at this value (call it T), the expected value of times the p-value is less than 0.05 is 0.05*(5) + 0.95*(T). 





## Python Programming

### 1

In [ ]:
# a
[2]*1000

In [ ]:
# b
[num%7 for num in range(1000)]

In [ ]:
# c
np.random.normal(10, 3, 1000)

### 2

In [ ]:
[sum(vec[x:x+7]) for x in range(0, len(vec), 7)]

### 3

In [ ]:
df = pd.read_csv('../hackerrank_data/skill-assessment/DSTakeHome/prog1.csv')
df[df['state']=='OH']['transactions'].mean()

## Machine Learning

### 1

Modeling a data as signal plus noise, underfitting is not capturing enough of the signal, whereas overfitting is capturing too much noise. 

This should be avoided because it means the model does not generalize as well to other subsamples where the noise is likely different. 

If your model does not generalize well to other independent datasets (drawn from the same population). One way to get a sense of this is through cross-validation---i.e., is there a large difference in the value of your scoring metric between your training and test sets.

### 2

Using K-means, perform the K-means algorithm for various values of k. For each, calculate the sum of squared error (i.e., the distance of each point to its final centroid, squared, and summed over all datas points). Plot this for each value of k and try to find an elbow. 

## Time Series

### 1

There is possibly a yearly seasonal component in the data. Only testing on the last month may not reveal whether the extent of the seasonality was captured by the model. The second method may have difficulties fully assessing parameter stability across years since you're training on multiple years. A possible improvement (in terms of robustness) is a type of cross-validation in which you train separately on each possible two year pairing (year 1 and year2, year 1 and year 3, year 2 and year3) and test against the data from the left out year (year 3, year 2, and year 1, respectively), taking the average (or min, or some other statistic) across the three folds. 



### 2

In [ ]:
# read data in
df_ts1 = pd.read_csv('../hackerrank_data/skill-assessment/DSTakeHome/ts1.csv')
df_ts2 = pd.read_csv('../hackerrank_data/skill-assessment/DSTakeHome/ts2.csv')
df_ts3 = pd.read_csv('../hackerrank_data/skill-assessment/DSTakeHome/ts3.csv')

df_ts1.date = pd.to_datetime(df_ts1.date)
df_ts1.index = df_ts1.date
df_ts1.drop(['date'], 1, inplace=True)

df_ts2.date = pd.to_datetime(df_ts2.date)
df_ts2.index = df_ts2.date
df_ts2.drop(['date'], 1, inplace=True)

df_ts3.date = pd.to_datetime(df_ts3.date)
df_ts3.index = df_ts3.date
df_ts3.drop(['date'], 1, inplace=True)

In [ ]:
# define a function that plots the series
def ts_plot(df):
    fig = plt.figure(figsize=(18,8))
    ax = fig.add_subplot(1,1,1)
    ax.plot(df)
    plt.show()

In [ ]:
ts_plot(df_ts1)
ts_plot(df_ts2)
ts_plot(df_ts3)

In [ ]:
# define a function that plots the acf and pcf
def acf_pcf_plot(df, lags):
    fig = plt.figure(figsize=(18,8))
    ax1 = fig.add_subplot(2,1,1)
    fig = plot_acf(df, lags=lags, ax=ax1)
    ax2 = fig.add_subplot(2,1,2)
    fig = plot_pacf(df, lags=lags, ax=ax2)
    plt.show()

#### ts1.csv

In [ ]:
acf_pcf_plot(df_ts1, 28) #shows a strong seasonal component, s=7

In [ ]:
df_ts1_diff = df_ts1.diff(periods=7).dropna() #take a seasonal difference
acf_pcf_plot(df_ts1_diff, 28) 

I would include the s=7 lag of the target variable as well as an SMA(1) term since there is a negative spike in the ACF at lag 7.

#### ts2.csv

In [ ]:
print df_ts2[df_ts2.target==0] #looks like the target is 0 on the 4th of July, Thanksgiving, and Christmas

acf_pcf_plot(df_ts2, 600) #shows a strong seasonal component, with at a lag of a year and half year

In [ ]:
#what does it look like after dropping the zero holidays?
df_ts2_noholi = df_ts2[df_ts2.target!=0]
acf_pcf_plot(df_ts2_noholi, 600) #don't see much if any correlation
# acf_pcf_plot(df_ts2_noholi, 100)

I throwout the zeros (which are perfectly explained by the holidays listed above) and then would include just a constant since there doesn't appear to be any MA or AR signatures and it seems to be stationary. 

#### ts3.csv

In [ ]:
df_ts3_diff = df_ts3.diff(periods=39).dropna() #seasonal differencing at frequency of 39 or 40 periods
ts_plot(df_ts3_diff)
acf_pcf_plot(df_ts3_diff, 100)

The plot of the seasonally differenced series still does not look stationary. Nevertheless, I'd add a 40 period lag and an AR term.

## 3

In [ ]:
# import the data and convert it to a series with the date as the index
df_ts4 = pd.read_csv('../hackerrank_data/skill-assessment/DSTakeHome/ts4.csv')
df_ts5 = pd.read_csv('../hackerrank_data/skill-assessment/DSTakeHome/ts5.csv')

df_ts4.date = pd.to_datetime(df_ts4.date)
df_ts4.index = df_ts4.date
df_ts4.drop(['date'], 1, inplace=True)

df_ts5.date = pd.to_datetime(df_ts5.date)
df_ts5.index = df_ts5.date
df_ts5.drop(['date'], 1, inplace=True)

df_ts4_test = pd.read_csv('../hackerrank_data/skill-assessment/DSTakeHome/ts4_test.csv')
df_ts4_test.date = pd.to_datetime(df_ts4_test.date)
df_ts4_test.index = df_ts4_test.date
df_ts4_test.drop(['date'], 1, inplace=True)

df_ts5_test = pd.read_csv('../hackerrank_data/skill-assessment/DSTakeHome/ts5_test.csv')
df_ts5_test.date = pd.to_datetime(df_ts5_test.date)
df_ts5_test.index = df_ts5_test.date
df_ts5_test.drop(['date'], 1, inplace=True)


In [ ]:
# Fourier transform function...x is the x is the data, n_predict is the number of predictions, and n_harm is the number of harmonics
def fourierExtrapolation(x, n_predict, n_harm):
    n = x.size
    t = np.arange(0, n)
    p = np.polyfit(t, x, 1)         # find linear trend in x
    x_notrend = x - p[0] * t        # detrended x
    x_freqdom = fft.fft(x_notrend)  # detrended x in frequency domain
    f = fft.fftfreq(n)              # frequencies
    indexes = range(n)
    # sort indexes by frequency, lower -> higher
    indexes.sort(key = lambda i: np.absolute(f[i]))
 
    t = np.arange(0, n + n_predict)
    restored_sig = np.zeros(t.size)
    for i in indexes[:1 + n_harm * 2]:
        ampli = np.absolute(x_freqdom[i]) / n   # amplitude
        phase = np.angle(x_freqdom[i])          # phase
        restored_sig += ampli * np.cos(2 * np.pi * f[i] * t + phase)
    return restored_sig + p[0] * t
    

### ts4.csv

In [ ]:
ts_plot(df_ts4)

In [ ]:
#the first differenced ts has lowest variance, suggesting first differencing is appropriate
print df_ts4.std()

df_ts4_diff = df_ts4.diff(periods=1).dropna()
print df_ts4_diff.std()

df_ts4_diff2 = df_ts4_diff.diff(periods=1).dropna()
print df_ts4_diff2.std()

acf_pcf_plot(df_ts4, 60) #also suggests first differencing

In [ ]:
# checking whether the original and first-differenced data measure stationary? yes
dftest = adfuller(df_ts4.target, autolag='AIC')
print dftest

dftest = adfuller(df_ts4_diff.target, autolag='AIC')
print dftest

In [ ]:
ts_plot(df_ts4_diff) #visually looks stationary

In [ ]:
acf_pcf_plot(df_ts4, 60)
acf_pcf_plot(df_ts4_diff, 60)

Based on these diagnostics, I fit three models to the training data: ARIMA(1,0,0), ARIMA(0,1,1), and a Fourier decomposition.

In [ ]:
# ARIMA(1,0,0)
model = ARIMA(df_ts4, (1,0,0)).fit(trend='c')
model1_pred = model.forecast(30)[0]

mae = np.mean(np.abs(df_ts4_test.target.values - model.forecast(30)[0])) #mean absolute error

print "ARIMA(1,0,0) MAE: {0}".format(mae)


In [ ]:
# ARIMA(0,1,1)
model = ARIMA(df_ts4, (0,1,1)).fit(trend='c')
model2_pred = model.forecast(30)[0]

mae = np.mean(np.abs(df_ts4_test.target.values - model.forecast(30)[0])) #mean absolute error

print "ARIMA(0,1,1) MAE: {0}".format(mae)


In [ ]:
# Fourier
n_predict = 30
extrapolation_ts4 = fourierExtrapolation(df_ts4.target.values, n_predict, 1)
mae = np.mean(np.abs(df_ts4_test.target.values - extrapolation_ts4[-30:]))
print "Fourier MAE: {0}".format(mae)


The latter scores lowest of the three. Writing these predictions to file...

In [ ]:
np.savetxt('ts4_pred.csv', extrapolation_ts4, delimiter=",")

Plotting the original series with the three predictions and the actual data...

In [ ]:
m1 = pd.DataFrame(model1_pred, columns=['target'])
m1.index = pd.date_range(start='2011-08-18', periods=30)

m2 = pd.DataFrame(model2_pred, columns=['target'])
m2.index = pd.date_range(start='2011-08-18', periods=30)

extrapolation_ts4_test = pd.DataFrame(extrapolation_ts4[-30:], columns=['target'])
extrapolation_ts4_test.index = pd.date_range(start='2011-08-18', periods=30)

fig = plt.figure(figsize=(18,8))
ax = fig.add_subplot(1,1,1)
ax.plot(df_ts4)
ax.plot(df_ts4_test, 'b', label='Actual')
ax.plot(m1, 'r', label='ARIMA(1,0,0)')
ax.plot(m2, 'k', label='ARIMA(0,1,1)')
ax.plot(extrapolation_ts4_test, 'g', label='Fourier Decomposition')
plt.legend()
plt.show()

### ts5.csv

In [ ]:
ts_plot(df_ts5)

In [ ]:
dftest = adfuller(df_ts5.target, autolag='AIC')
print dftest

In [ ]:
acf_pcf_plot(df_ts5, 60)

Based on these preliminary diagnostics, the series looks a lot like white noise. I'll predict with the mean and a Fourier decomposition.

In [ ]:
# mean
model1_pred = [float(df_ts5.mean())]*30

mae = np.mean(np.abs(df_ts5_test.target.values - model1_pred)) #mean absolute error

print "MAE of the mean: {0}".format(mae)

In [ ]:
# Fourier
n_predict = 30
extrapolation_ts5 = fourierExtrapolation(df_ts5.target.values, n_predict, 3)

mae = np.mean(np.abs(df_ts5_test.target.values - extrapolation_ts5[-30:]))
print "Fourier MAE: {0}".format(mae)

The latter scores lowest of the two. Writing these predictions to file...

In [ ]:
np.savetxt('ts5_pred.csv', extrapolation_ts5, delimiter=",")

Plotting the original series with the two predictions and the actual data...

In [ ]:
m1 = pd.DataFrame(model1_pred, columns=['target'])
m1.index = pd.date_range(start='2016-08-31', periods=30)

extrapolation_ts5_test = pd.DataFrame(extrapolation_ts5[-30:], columns=['target'])
extrapolation_ts5_test.index = pd.date_range(start='2016-08-31', periods=30)

fig = plt.figure(figsize=(18,8))
ax = fig.add_subplot(1,1,1)
ax.plot(df_ts5)
ax.plot(df_ts5_test, 'b', label='Actual')
ax.plot(m1, 'r', label='Mean')
ax.plot(extrapolation_ts5_test, 'g', label='Fourier Decomposition')
ax.set_ylim([145, 155])
plt.legend(loc=2)
plt.show()